# 제29회 TEM 워크샵 머신러닝 실습
이번 시간에는 딥러닝 모델을 TEM 데이터에 적용하기 전에 일반적인 이미지를 다루는 방법과 딥러닝 모델을 설계하고 학습하는 방법을 실습을 통해 배울 예정입니다.

## 목차
### I. MNIST 데이터 분류

#### I-1. 데이터 살펴보기

#### I-2. 모델 구축하기

#### I-3. 모델 학습하기

#### I-4. 모델 검증하기

### II. CIFAR100 데이터 분류

#### II-1. 데이터 살펴보기

#### II-2. 모델 구축하기

#### II-3. 모델 학습하기

#### II-4. 모델 검증하기

## I. MNIST 데이터 분류
MNIST 데이터는 0에서 9 사이의 숫자 손글씨 데이터와 라벨로 구성된 데이터로 딥러닝 입문에서 가장 많이 사용하는 데이터셋 중 하나입니다.

이번 파트에서는 MNIST 데이터를 이용해 이미지 데이터를 전처리 하는 방법, 딥러닝 모델을 구축하는 방법, 딥러닝 모델을 이용하여 데이터를 분류하는 방법, 마지막으로 학습된 모델을 검증하는 방법까지 배워보겠습니다.

### I-1. 데이터 살펴보기

우선 이번 실습에서 필요한 library들을 불러옵니다.

torch는 딥러닝 분야에서 가장 많이 사용되는 PyTorch library이며, 딥러닝 모델 구축 및 학습에 필요한 데이터셋, layer, optimizer 등을 포함하고 있습니다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn # layers (convolution, linear, pooling, etc.)
import torch.optim as optim # optimizers (SGD, Adam)
from torchvision import datasets # 데이터셋 (MNIST, CIFAR100)
from torchvision import transforms # 데이터 전처리

---
이제 저희가 사용할 MNIST 데이터를 불러옵니다.

모델 학습에 필요한 training data와 test data로 나누어져 있으며, 처음 불러올 경우 다운로드가 필요합니다.

In [ ]:
transform = transforms.ToTensor() # PIL 이미지를 torch의 tensor로 변환

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform) # training data
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform) # test data

---
데이터셋의 크기를 확인해 봅시다.

출력 순서는 [데이터 수, 높이, 너비] 입니다.

In [ ]:
# tensor.size(): tensor의 크기를 반환
print("Size of training data: ", train_data.data.size())
print("\nSize of test data: ", ### 코드 작성 ###)

---
MNIST 데이터가 어떤 class로 이루어져 있는지, 그리고 라벨은 어떻게 구성되어 있는지 살펴봅시다.

In [ ]:
print("Classes:", test_data.classes)
print("\nLabels:", test_data.targets)

---
다음으로 MNIST 데이터의 이미지들이 어떻게 구성되어 있는지 살펴봅시다.

아래 코드를 실행하면 0에서 9까지의 이미지를 각각 10장씩 출력합니다.

In [ ]:
fig, ax = plt.subplots(len(test_data.classes), 10, figsize=(10, 10))

for img_class, row in enumerate(ax):
    class_idx = np.where(np.array(test_data.targets) == img_class)[0]
    for i, plot in enumerate(row):
        if i == 0:
            plot.set_ylabel(test_data.classes[img_class])
        plot.set_yticks([])
        plot.set_xticks([])
        idx = class_idx[i]
        img = test_data.data[idx]
        plot.imshow(img, cmap='gray')
plt.tight_layout()

---
데이터 살펴보기의 마지막 단계로 DataLoader를 만들어 봅시다.

먼저 **batch**라는 개념에 대해 알아야 합니다.

딥러닝 모델을 학습시킬 때 한 번의 가중치 업데이트를 위해 여러 개의 training data를 묶어서 사용하게 되는데, 이러한 묶음을 **batch**라고 합니다.

**batch size**는 한 batch에 들어가는 데이터의 수를 의미합니다.

예를 들어 데이터 수가 1000개라고 할 때 batch size가 10이면 100 개의 batch, batch size가 100이면 10개의 batch로 데이터셋이 나누어지게 됩니다.

<center>
<img src = "https://drive.google.com/uc?id=1mQOfmfIkdDNlBRrs9EWut8EM26lprzmz" width="700" /><br>
</center>

DataLoader는 데이터셋을 batch 단위로 나눠주고 매번 데이터 순서를 바꿔주는 등의 역할을 합니다.

In [ ]:
batch_size = 32

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True) # training data는 매번 데이터 순서를 섞어줌 (shuffle=True)
test_loader = ### 코드 작성 ###

### I-2. 모델 구축하기

이제 본격적으로 모델을 구축하는 방법에 대해 알아보겠습니다.

우선 모델 구축에 필요한 layer들은 torch.nn library에 class 형태로 포함되어 있습니다.

저희가 사용할 layer는 convolution, max pooling, linear, 그리고 ReLU activation 입니다.

### 1) Convolution
2D Convolution은 CNN의 핵심 연산으로 이미지 데이터를 다룰 때 가장 많이 사용하는 layer입니다.

Convolution layer를 이용해 이미지에서 중요한 특징(feature)을 추출할 수 있으며, 아래와 같이 사용합니다.

`nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias)`
- in_channels: 입력 데이터 채널의 개수입니다. 예를 들어 MNIST와 같이 흑백 이미지면 1, 컬러 이미지면 3(RGB)이 됩니다.
- out_channels: 출력 데이터 채널의 개수입니다. 이는 convolution filter의 개수와 동일합니다.
- kernel_size: convolution filter의 크기 입니다.
- stride: filter의 이동 간격입니다. stride가 커지면 출력 데이터의 크기가 작아집니다.
- padding: 입력 데이터 가장자리에 추가되는 패딩의 크기입니다. 패딩을 추가함으로써 출력 데이터의 크기를 보존할 수 있습니다.
- bias: bias를 사용할지 말지에 대한 여부입니다.


---
아래 예시 코드는 8x8 크기의 3 채널 이미지를 convolution layer에 통과시키는 코드입니다.

출력 데이터의 사이즈는 아래 식과 같으며, 여러 파라미터를 바꾸며 출력 데이터의 크기를 확인해보세요.

output size = (input size - kernel_size + 2 * padding) / stride + 1
<center>
<img src = "https://drive.google.com/uc?id=1L7m0wqLDOU61hEJgytLbbDpGM6QWsl7A" width="900" /><br>
</center>

In [ ]:
# 입력 데이터 크기 (batch size, channels, height, width)
input_size = (4, 3, 8, 8)

# 입력 데이터 생성
input_data = torch.randn(input_size)
print("Input size: ", input_data.size())

# Convolution layer 정의
conv = ### 코드 작성 ###

# Convolution layer 통과하여 출력 데이터 생성
output_data = conv(input_data)

# 출력 데이터 크기
print("Output size: ", output_data.size())

### 2) Max Pooling
Max pooling은 feature map의 크기를 줄이는 연산들 중 하나이며, window 내에 있는 값들 중 가장 큰 값을 선택하여 정보를 압축합니다.

Max pooling layer는 convolutin layer와 비슷하게 아래와 같이 사용합니다.

`nn.MaxPool2d(kernel_size, stride, padding)`
- kernel_size: pooling window의 크기 입니다.
- stride: window의 이동 간격입니다. 일반적으로 kernel_size와 같게 설정합니다.
- padding: 입력 데이터 가장자리에 추가되는 패딩의 크기입니다.

---
아래 예시 코드는 4x4 크기의 이미지를 max pooling layer에 통과시키는 코드입니다.

출력 데이터의 사이즈는 convolution과 동일하게 계산할 수 있으며, 여러 파라미터를 바꾸며 출력 데이터의 크기를 확인해보세요.

<center>
<img src = "https://drive.google.com/uc?id=1qqeKnmro8vhx_aNRXb7x-h671j34C5d8" width="800" /><br>
</center>

In [ ]:
# 입력 데이터 생성 및 크기 확인
input_data = np.array([[1, 2, 8, 5], [4, 3, 7, 6], [11, 12, 14, 15], [10, 9, 13, 16]], dtype=np.float32)
input_data = torch.from_numpy(input_data).unsqueeze(0).unsqueeze(0)
print("Input size: ", input_data.size())
print("Input data:\n", input_data)

# Max pooling layer 정의
pool = ### 코드 작성 ###

# Max pooling layer 통과하여 출력 데이터 생성
output_data = pool(input_data)

# 출력 데이터 확인
print("\nOutput size: ", output_data.size())
print("Output data:\n", output_data)

### 3) Linear
Linear는 linear transformation의 역할을 하는 layer로 fully connected layer, dense layer 등으로도 불립니다.

Linear layer는 아래와 같이 사용합니다.

`nn.Linear(in_features, out_features, bias)`
- in_features: 입력 feature의 개수입니다.
- out_features: 출력 feature의 개수입니다.
- bias: bias를 사용할지 말지에 대한 여부입니다.

---
아래 예시 코드는 linear layer의 사용 예시입니다.

<center>
<img src = "https://drive.google.com/uc?id=1E3_DIc-PxfTlbsdqucnNbUDkiwPI6nVP" width="700" /><br>
</center>

In [ ]:
# 입력 데이터 생성 (batch size, in_features)
input_data = torch.randn(4, 5)
print("Input size: ", input_data.size())

# LInear layer 정의
fc = ### 코드 작성 ###

# Convolution layer 통과하여 출력 데이터 생성
output_data = fc(input_data)

# 출력 데이터 크기
print("Output size: ", output_data.size())

### 4) ReLU
ReLU는 activation function 중 하나로 input 값과 0중 큰 값을 반환합니다.

즉, ReLU Layer를 통과하면 input 값 중 양수 값만 남고 나머지는 0이 됩니다.

ReLU layer는 아래와 같이 사용합니다.

`nn.ReLU()`

---
아래 예시 코드는 ReLU layer의 사용 예시입니다.

<center>
<img src = "https://drive.google.com/uc?id=1uRboEdV3plVjVi_lIP2zWMNAm3930PSI" width="700" /><br>
</center>

In [ ]:
# 입력 데이터 생성 확인
input_data = np.array([[0.7, -1.3, 2.3, 5.6, -3.8]], dtype=np.float32)
input_data = torch.from_numpy(input_data)
print("Input data:\n", input_data)

# ReLU layer 정의
relu = ### 코드 작성 ###

# ReLU layer 통과하여 출력 데이터 생성
output_data = relu(input_data)

# 출력 데이터 확인
print("Output data:\n", output_data)

---
이제 앞서 나온 layer들을 이용하여 실제로 사용할 모델을 만들어보겠습니다.

저희가 만들 모델은 아래와 같이 CNN을 이용해서 이미지에서 feature를 추출하고, 추출된 feature를 fully connected layer에 통과시켜 모델이 이미지의 class를 예측하는 구조입니다.

모델의 최종 출력은 해당 이미지가 각 class일 확률을 의미합니다.

(e.g. 0일 확률 0.1, 1일 확률 0.4, ..., 9일 확률 0 -> [0.1, 0.4, ..., 0])

** 정확히는 확률이 아닌 logit이며 이는 뒤에서 추가 설명
<center>
<img src = "https://drive.google.com/uc?id=1kkFoWtZRmc2uhLPxnr4icwqO8iPGd8He" width="800" /><br>
</center>

---
torch library를 이용한 딥러닝 모델들은 nn.Module의 하위 클래스를 생성하여 만들 수 있습니다.

`__init__` method에서 모델에 사용할 layer들을 정의해주고, `__forward__` method에서는 input이 들어왔을 때 앞서 정의한 layer들을 통과하여 최종 output을 return하도록 모델을 작성합니다.

In [ ]:
class MODEL(nn.Module): # nn.Module의 하위 클래스
    # img_channels: input image의 channel 수, MNIST 데이터는 1
    # conv_num_features1: 첫 번째 convolution layer의 output channel 수
    # conv_num_features2: 두 번째 convolutino layer의 output channel 수
    # fc_num_features: 첫 번째 fully connected layer의 ouptut feature 수
    # num_class: class의 개수이자 마지막 fully connected layer의 output feature 수, MNIST 데이터는 10
    def __init__(self, img_channels, conv_num_features1, conv_num_features2, fc_num_features, num_class):
        super(MODEL, self).__init__()

        # 첫 번째 convolution layer, kernel_size=3
        self.conv1 = nn.Conv2d(in_channels = img_channels,
                               out_channels = conv_num_features1,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1,
                               bias = True)
        # 두 번째 convolutin layer, kernel_size=3
        ### 코드 작성 ###

        # 첫 번째 fully connected layer
        self.fc1 = nn.Linear(conv_num_features2 * 7 * 7, fc_num_features)
        # 두 번째 fully connected layer
        ### 코드 작성 ###

        # ReLU activation
        self.relu = nn.ReLU()
        # Max Pooling
        self.maxpool = nn.MaxPool2d(2, 2)


    def forward(self, x):
        # x.size() = (B, 1, 28, 28)
        x = self.conv1(x) # (B, 1, 28, 28) -> (B, C1, 28, 28)
        x = self.relu(x)
        x = self.maxpool(x) # (B, C1, 28, 28) -> (B, C1, 14, 14)

        x = ### 코드 작성 ### # (B, C1, 14, 14) -> (B, C2, 14, 14)
        x = ### 코드 작성 ###
        x = ### 코드 작성 ### # (B, C2, 14, 14) -> (B, C2, 7, 7)

        # tensor.view(shape): tensor를 원하는 shape으로 reshape, -1은 자동으로 shape을 지정
        x = x.view(len(x), -1) # (B, C2, 7, 7) -> (B, C2x7x7)

        x = self.fc1(x) # (B, C2x7x7) -> (B, C3)
        x = self.relu(x)

        x = ### 코드 작성 ### # (B, C3) -> (B, 10)
        return x

---
이제 모델에 사용할 hyperparameter를 설정해줍니다.

MNIST 데이터에 맞게 `img_channels`와 `num_class`는 각각 1, 10으로 설정해주고 각 layer의 channel/feature수도 적당한 값으로 설정합니다.

일반적으로 channel/feature 수는 2의 거듭제곱으로 지정합니다.

In [ ]:
img_channels = 1
conv_num_features1 = 32
conv_num_features2 = 64
fc_num_features = 64
num_class = 10

---
cpu와 gpu (cuda) 중 어떤 device를 사용할지 아래와 같이 정의합니다.

또한 앞서 설정한 hyperparameter로 모델을 정의합니다.

In [ ]:
# device 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 정의
model = ### 코드 작성 ###
model = model.to(device)

### I-3. 모델 학습하기

이제 모델을 학습할 차례입니다.

학습에 앞서 몇 가지 hyperparameter를 설정합니다.

Epoch은 학습 데이터를 총 몇 번 반복하여 학습할 것인지 결정하는 hyperparameter입니다.

Learning rate는 모델의 가중치(weight) 업데이트의 정도를 결정하는 hyperparameter입니다.

Learning rate가 크면 loss가 빠르게 수렴하지만 학습이 불안정할 수 있으며, 반대로 작을 경우 학습은 안정적이지만 최적값에 다다르기까지 시간이 매우 오래 걸릴 수 있으므로 적절한 값으로 설정해줍니다.


In [ ]:
# Epoch 수 설정
EPOCH = 10
# Learning rate 설정
learning_rate = 0.001

---
다음으로 optimizer와 loss를 정의해줍니다.

Optimizer는 일반적으로 많이 사용하는 Adam optimizer를 사용하며, 업데이트할 parameter와 learning rate를 input으로 받습니다.

---
Loss는 classification에서 사용하는 cross entropy loss를 사용하며, 아래와 같은 수식으로 정의됩니다.

$$H(P, Q) = -∑p_i\log(q_i)$$

여기서 𝑃 는 one hot 형태의 label 이며 𝑄는 모델이 예측한 확률이라고 생각할 수 있습니다.

e.g. label = 5 ->

𝑃=[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

𝑄=[0.1, 0, 0, 0, 0.7, 0.2, 0, 0, 0]

𝐻=-log(0.7)=0.3567

---
따라서 cross entropy를 계산하기 위해서는 label을 모두 one hot 형태로 바꿔주고 모델의 출력인 logit을 softmax를 이용하여 확률값으로 바꿔주어야 합니다.

하지만 `torch.nn`에서 제공하는 `CrossEntropyLoss`를 사용할 경우 이를 자동으로 수행해줍니다.

In [ ]:
# Optimizer 정의
optimizer = optim.Adam(params = model.parameters(), lr = learning_rate)

# Loss function 정의
criterion = nn.CrossEntropyLoss()

---
이제 아래 코드를 통해 모델을 직접 학습해 봅시다.

In [ ]:
# Epoch 수만큼 학습 반복
for epoch in range(1, EPOCH + 1):
    print(f"EPOCH : {epoch}")

    # 각 epoch 별 loss와 accuracy
    train_loss = 0.0
    train_acc = 0.0

    for images, labels in train_loader:
        # 학습에 사용할 tensor들은 모두 device로 보내줌
        images = images.to(device)
        labels = labels.to(device)

        # Model의 output은 logit
        logits = model(images)

        # Cross entropy loss 계산
        loss = criterion(logits, labels)

        # Optimizer gradient 초기화
        optimizer.zero_grad()

        # Gradient 계산
        loss.backward()

        # Weight update
        optimizer.step()

        train_loss += loss.item() * images.size(0)

        _, prediction = logits.max(1)

        train_acc += (labels == prediction).sum().item()

    train_loss = train_loss / len(train_loader.dataset)
    train_acc = train_acc / len(train_loader.dataset) * 100
    print("| Training Loss : {:.3f} | Training Accuracy: {:.3f} % |".format(train_loss, train_acc))

### I-4. 모델 검증하기

마지막으로 학습된 모델을 검증해보겠습니다.

모델 검증은 학습에 사용되지 않은, 즉 모델이 보지 못한 test data를 이용하여 진행합니다.

아래 코드를 실행시키면 test data에 대한 accuracy를 구할 수 있습니다.

In [ ]:
model.eval()

test_acc = 0
predictions = []
for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)

    logits = model(images)

    _, prediction = logits.max(1)

    test_acc += (labels == prediction).sum().item()

    predictions.append(prediction.detach().cpu().numpy())

predictions = np.concatenate(predictions)

test_acc = test_acc / len(test_loader.dataset) * 100
print("| Test Accuracy: {:.3f} % |".format(test_acc))

---
각 class 별로 10개의 이미지와 그에 대해 모델이 예측한 class를 출력하여 확인해봅니다.

In [ ]:
fig, ax = plt.subplots(len(test_data.classes), 10, figsize=(10, 10))

for img_class, row in enumerate(ax):
    class_idx = np.where(np.array(test_data.targets) == img_class)[0]
    for i, plot in enumerate(row):
        if i == 0:
            plot.set_ylabel(test_data.classes[img_class])
        plot.set_yticks([])
        plot.set_xticks([])
        idx = class_idx[i]
        plot.set_xlabel(predictions[idx])
        img = test_data.data[idx]
        plot.imshow(img, cmap='gray')
plt.tight_layout()

---
MNIST data classification은 마무리 되었습니다.

시간 여유가 있으시다면 여러 hyperparameter(batch size, learning rate, epoch 등)들을 바꿔서 결과를 비교해보세요.

## II. CIFAR100 데이터 분류
CIFAR100 데이터는 사과, 물고기, 아기 등 총 100가지 class의 컬러 이미지들로 구성된 데이터입니다.

이번 파트에서는 MNIST 데이터보다 좀 더 복잡한 CIFAR100 데이터를 이용해 classification을 진행해 보겠습니다.

### II-1. 데이터 살펴보기

MNIST 데이터와 마찬가지로 데이터를 다운로드하고 불러옵니다.

조금 다른 점은 train data에 대해서는 무작위로 좌우 반전 (`transforms.RandomHorizontalFlip()`)을 적용하여 data augmentation 효과를 줍니다.

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
])

train_data = datasets.CIFAR100(root='./data', train=True, download=True, transform=train_transform)
test_data = datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transform)

---
데이터셋의 크기를 확인해 봅시다.
출력 순서는 [데이터 수, 높이, 너비, 채널 수] 입니다.

In [ ]:
print("Size of training data: ", train_data.data.shape)
print("\nSize of test data: ", test_data.data.shape)

---
CIFAR100 데이터가 어떤 class를 포함하고 있는지 확인합니다.

In [ ]:
print("Classes:", test_data.classes)

아래 코드를 실행하여 100개의 class 중 10개의 class에 대해서 각 10장씩 이미지를 확인해 봅시다.

In [ ]:
fig, ax = plt.subplots(10, 10, figsize=(10, 10))

for img_class, row in enumerate(ax):
    class_idx = np.where(np.array(test_data.targets) == img_class)[0]
    for i, plot in enumerate(row):
        if i == 0:
            plot.set_ylabel(test_data.classes[img_class])
        plot.set_yticks([])
        plot.set_xticks([])
        idx = class_idx[i]
        img = test_data.data[idx]
        plot.imshow(img, cmap='gray')
plt.tight_layout()

---
Batch size를 설정하고 DataLoader를 만들어줍니다.

In [ ]:
batch_size = 32

train_loader = ### 코드 작성 ###
test_loader = ### 코드 작성 ###

### II-2. 모델 구축하기

MNIST와 마찬가지로 CIFAR100 데이터를 위한 모델을 구축해 봅시다.

앞에서 사용한 모델과 거의 똑같지만 한 가지 다른 점은 convolution layer를 추가하여 더 깊은 CNN 구조를 사용한다는 것입니다.
<center>
<img src = "https://drive.google.com/uc?id=1SPkFGnHxNH8Y7YpuS7dNm-veRw_dygBI" width="900" /><br>
</center>

In [ ]:
class MODEL(nn.Module):
    # img_channels: input image의 channel 수, MNIST 데이터는 1
    # conv_num_features1: 첫 번째 convolution layer의 output channel 수
    # conv_num_features2: 두 번째 convolutino layer의 output channel 수
    # conv_num_features3: 세 번째 convolutino layer의 output channel 수
    # fc_num_features: 첫 번째 fully connected layer의 ouptut feature 수
    # num_class: class의 개수이자 마지막 fully connected layer의 output feature 수, MNIST 데이터는 10
    def __init__(self, img_channels, conv_num_features1, conv_num_features2, conv_num_features3, fc_num_features, num_class):
        super(MODEL, self).__init__()

        # 첫 번째 convolution layer, kernel_size=3
        self.conv1 = ### 코드 작성 ###
        # 두 번째 convolutin layer, kernel_size=3
        self.conv2 = ### 코드 작성 ###
        # 세 번째 convolutin layer, kernel_size=3
        self.conv3 = ### 코드 작성 ###

        # 첫 번째 fully connected layer
        self.fc1 = ### 코드 작성 ###
        # 두 번째 fully connected layer
        self.fc2 = ### 코드 작성 ###

        # ReLU activation
        self.relu = nn.ReLU()
        # Max Pooling
        self.maxpool = nn.MaxPool2d(2, 2)


    def forward(self, x):
         # x.size() = (B, 3, 32, 32)
        x = ### 코드 작성 ### # (B, 3, 32, 32) -> (B, C1, 32, 32)
        x = ### 코드 작성 ###
        x = ### 코드 작성 ### # (B, C1, 32, 32) -> (B, C1, 16, 16)

        x = ### 코드 작성 ### # (B, C1, 16, 16) -> (B, C2, 16, 16)
        x = ### 코드 작성 ###
        x = ### 코드 작성 ### # (B, C2, 16, 16) -> (B, C2, 8, 8)

        x = ### 코드 작성 ### # (B, C2, 8, 8) -> (B, C3, 8, 8)
        x = ### 코드 작성 ###
        x = ### 코드 작성 ### # (B, C3, 8, 8) -> (B, C3, 4, 4)

        x = x.view(len(x), -1) # (B, C3, 4, 4) -> (B, C3x4x4)

        x = ### 코드 작성 ### # (B, C3x4x4) -> (B, C4)
        x = ### 코드 작성 ###

        x = ### 코드 작성 ### # (B, C4) -> (B, 100)
        return x

---
Hyperparameter를 설정해줍니다.

CIFAR100 이미지는 컬러 이미지이므로 `img_channels`는 3 (RGB), `num_class`는 100으로 설정해줍니다.

In [ ]:
img_channels = 3
conv_num_features1 = 32
conv_num_features2 = 64
conv_num_features3 = 128
fc_num_features = 128
num_class = 100

---
Device와 모델도 정의해줍니다.

In [ ]:
# device 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 정의
model = ### 코드 작성 ###
model = model.to(device)

### II-3. 모델 학습하기

모델 학습은 이전과 동일하게 hyperparameter를 설정해주고 진행합니다.

In [ ]:
# Epoch 수 설정
EPOCH = 100
# Learning rate 설정
learning_rate = 0.001

---
마찬가지로 Adam optimizer와 cross entropy loss를 사용합니다.

In [ ]:
# Optimizer 정의
optimizer = optim.Adam(params = model.parameters(), lr = learning_rate)

# Loss function 정의
criterion = nn.CrossEntropyLoss()

---
모델을 학습합니다.

In [ ]:
for epoch in range(1, EPOCH + 1):
    print(f"EPOCH : {epoch}")

    train_loss = 0.0
    train_acc = 0.0

    for images, labels in train_loader:
        ### 코드 작성 ###

        train_loss += loss.item() * images.size(0)

        _, prediction = logits.max(1)

        train_acc += (labels == prediction).sum().item()

    train_loss = train_loss / len(train_loader.dataset)
    train_acc = train_acc / len(train_loader.dataset) * 100
    print("| Training Loss : {:.3f} | Training Accuracy: {:.3f} % |".format(train_loss, train_acc))

### II-4. 모델 검증하기

마찬가지로 test set을 이용해 모델을 검증해봅시다.

앞서 MNIST 데이터보다 복잡한 CIFAR100 데이터를 사용했을 때 결과가 어떻게 달라졌는지 비교해봅니다.

In [ ]:
model.eval()

test_acc = 0
predictions = []
for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)

    logits = model(images)

    _, prediction = logits.max(1)

    test_acc += (labels == prediction).sum().item()

    predictions.append(prediction.detach().cpu().numpy())

predictions = np.concatenate(predictions)

test_acc = test_acc / len(test_loader.dataset) * 100
print("| Test Accuracy: {:.3f} % |".format(test_acc))

---
MNIST 데이터와 달리 training accuracy에 비해 test accuracy가 아주 낮게 나오는 것을 볼 수 있습니다.

이는 모델이 training data의 정보만을 외워서 학습하여 다른 데이터가 들어왔을 때 제대로 예측하지 못하기 때문입니다.

이러한 현상을 과적합(overfitting)이라고 합니다.

이를 방지하기 위해서는 여러 hyperparameter의 조절, data augmentation, 별도의 validation data을 이용한 ealry stopping 등이 필요합니다.


---
각 class 별로 10개의 이미지와 그에 대해 모델이 예측한 class를 출력하여 확인해봅니다.

In [ ]:
fig, ax = plt.subplots(10, 10, figsize=(10, 10))

for img_class, row in enumerate(ax):
    class_idx = np.where(np.array(test_data.targets) == img_class)[0]
    for i, plot in enumerate(row):
        if i == 0:
            plot.set_ylabel(test_data.classes[img_class])
        plot.set_yticks([])
        plot.set_xticks([])
        idx = class_idx[i]
        plot.set_xlabel(test_data.classes[predictions[idx]])
        img = test_data.data[idx]
        plot.imshow(img, cmap='gray')
plt.tight_layout()

---
이로써 CIFAR100 data classification도 마무리되었습니다.

다른 hyperparameter를 바꾸었을 때 결과가 어떻게 달라지는지 확인해보세요.